In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math 

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
@pyimport numpy
@pyimport PIL
@pyimport skimage.transform as skimage_transform
@pyimport sklearn.cluster as cluster
@pyimport matplotlib.cm as cm

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
corner_cue = 
[
    ["20220406_111526", 9, "20220406_125842", 9],
    ["20220407_090156", 5, "20220407_104712", 5],
    ["20220406_153842", 9, "20220406_171558", 9]
]


In [ ]:
data_all_exp = reduce(vcat, [corner_cue])

In [ ]:
data_all = []
server_all = []
for data in data_all_exp
    if length(data).== 4
        append!(data_all, [data[1]])
        append!(data_all, [data[3]])
        append!(server_all, data[2])
        append!(server_all, data[4])
    elseif length(data).== 2
        append!(data_all, [data[1]])
        append!(server_all, data[2])
    end
end

In [ ]:
which_combo = 2
experimenter = "jen"

In [ ]:
experiment_filename_1 = data_all_exp[which_combo][1]
server = data_all_exp[which_combo][2]
ds_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_1 = Dataset(experiment_filename_1, "jen", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")

experiment_filename_2 = data_all_exp[which_combo][3]
server = data_all_exp[which_combo][4]
ds_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_2 = Dataset(experiment_filename_2, "jen", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")

In [ ]:
@pyimport matplotlib.patches as patches

In [ ]:
n_pos = 60

In [ ]:
# orientation-corrected fish location (time binned)
position_file = h5open(joinpath(data_path(ds_save_cy_1), "oblique_corrected", "for_place_calculation_chamber_geometry_$(experiment_filename_1)_n60_oblique_corrected.h5"))
x_fish_sweep_mean = read(position_file,"x_fish_sweep_mean")
y_fish_sweep_mean = read(position_file,"y_fish_sweep_mean")
mask_valid = read(position_file,"mask_valid")
x_digital = read(position_file,"x_digital")
y_digital = read(position_file,"y_digital")
x_bins = read(position_file,"x_bins")
y_bins = read(position_file,"y_bins")
# chamber_roi = read(position_file,"chamber_roi")
what_angle = read(position_file,"what_angle")
close(position_file)
mask_invalid = .!mask_valid;
loc_digital = (y_digital.-1).*n_pos.+x_digital;

In [ ]:
ds_save_cy_roi = Dataset("20220406_111526", "chuyu", gethostname() == "roli-$(9)" ? "/data" : "/nfs/data$(9)")
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_roi), "old/orientation_correction.h5"))
chamber_roi = read(orientation_correction_file,"chamber_roi_r")
close(orientation_correction_file)

chamber_roi[chamber_roi.!=0].=1;

In [ ]:
img_bg_1 = h5open(ds_save_1, "behavior.h5"; raw = true) do file
    read(file, "img_bg")
end;

global img_bg_end_1 = img_bg_1[:,:,end]

global w = size(img_bg_end_1, 1)
global l = size(img_bg_end_1, 2)

In [ ]:
NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
global Z_all = HDF5.readmmap(NMF_file["Z_all"])
global X_all = HDF5.readmmap(NMF_file["X_all"])
global Y_all = HDF5.readmmap(NMF_file["Y_all"])
global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
global A_dFF_1 = HDF5.readmmap(NMF_file["A_dF"]); # bs update
global A_baseline_1 = HDF5.readmmap(NMF_file["A_baseline"]); # bs update
close(NMF_file)


n_neuron = length(X_all);

In [ ]:
file_folder_1 = joinpath(data_path(ds_save_cy_1), "oblique_corrected")
all_files = readdir(file_folder_1)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = [occursin("A_dF", long_name_files[i])*occursin("neuron", long_name_files[i]) for i in 1:length(long_name_files)]
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(file_folder_1, save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
specificity_1 = HDF5.readmmap(file["specificity"])
specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
plot_index = HDF5.readmmap(file["bool_index"])
close(file)

In [ ]:
# match the frames if we have offsets
n_neurons = size(A_dFF_1, 2)
n_sweeps = size(A_dFF_1, 1)

x_fish_sweep_mean = x_fish_sweep_mean[1:n_sweeps]
y_fish_sweep_mean = y_fish_sweep_mean[1:n_sweeps];

In [ ]:
plot(x_fish_sweep_mean)

In [ ]:

chamber_roi_mask = Float32.(copy(chamber_roi))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32;


img_bg_mask = copy(img_bg_end_1)
img_bg_mask[img_bg_mask.==0].=NaN32
img_bg_mask[Float32.(chamber_roi).!=0].=NaN32;

In [ ]:
imshow(chamber_roi_mask)

In [ ]:
imshow(chamber_roi)

chamber_roi[chamber_roi.!=0].=1;
countour = find_contours(chamber_roi)[1];
countour_array = hcat(getindex.(countour, 1), getindex.(countour,2));

In [ ]:
good_candidates = [62717, 9097, 34019, 62722, 63668, 27105]

In [ ]:
mask_valid = isfinite.(place_map_all_1[:,:,1])

mask_valid_index = findall(mask_valid)
mask_valid_x = [x[1] for x in mask_valid_index]
mask_valid_y = [x[2] for x in mask_valid_index];
mask_valid_x_min = minimum(mask_valid_x)
mask_valid_x_max = maximum(mask_valid_x)
mask_valid_y_min = minimum(mask_valid_y)
mask_valid_y_max = maximum(mask_valid_y);

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/figure1/place_cell_examples"
mkpath(fig_path)

In [ ]:
# example cells

valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

for real_cell_index in good_candidates
    fig = figure()
    example_map = place_map_all_1[mask_valid_x_min:mask_valid_x_max,mask_valid_y_min:mask_valid_y_max,real_cell_index]
    example_map = (example_map.-nanmean(example_map))./nanstd(example_map)
    sp1 = imshow(example_map', extent = [x_bins[mask_valid_x_min], x_bins[mask_valid_x_max+1], y_bins[mask_valid_y_min], y_bins[mask_valid_y_max+1]], cmap="jet", origin="lower")
    imshow(chamber_roi_mask', cmap="binary", origin="lower")
    axis("off")
    xlim(xmin, xmax)
    ylim(ymax, ymin)
    tight_layout()
    fig.savefig(joinpath(fig_path, "$(experiment_filename_1)_$real_cell_index map1.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end

In [ ]:
function activity_trajectory(neural_activity, x_fish_valid, y_fish_valid,alpha=1)
    """
    neural_activity: activity in the chosen period
    x_fish_valid: fish location x in the chosen period
    y_fish_valid: fish location y in the chosen period
    alpha : alpha for the background
    """
    fig = figure(dpi=250)
    plot(x_fish_valid, y_fish_valid, "k.", markersize = 0.1)
    
    layer_rank = sortperm(neural_activity[1:end-1])
    for i = layer_rank
        cur_value = neural_activity[i]; 
        cur_value < 0 ? cur_value = 0 : (cur_value > 1 ? cur_value = 1 : 1+1)
        if isfinite(cur_value)
            plot(x_fish_valid[i:i+1], y_fish_valid[i:i+1], color = [cur_value, 0, 0], alpha = 0.5, linewidth = 2)
        else
            plot(x_fish_valid[i:i+1], y_fish_valid[i:i+1], color = [0, 0, 0], alpha = 0.1)
        end

    end
    imshow(img_bg_end_1', cmap="binary", origin="lower",alpha=alpha)
    axis("off")
    return fig
end

In [ ]:
for real_cell_index in good_candidates
    neural_activity = A_dFF_1[plot_index, real_cell_index]
    neural_activity = (neural_activity.-numpy.nanmean(neural_activity))./numpy.nanstd(neural_activity)/3;
    x_fish_valid = x_fish_sweep_mean[plot_index]
    y_fish_valid = y_fish_sweep_mean[plot_index]
    fig = activity_trajectory(neural_activity, x_fish_valid, y_fish_valid,0)
    xlim(xmin, xmax)
    ylim(ymax, ymin)
    axis("off")
    fig.savefig(joinpath(fig_path,  "$(experiment_filename_1)_$real_cell_index traj1.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end